# RNNs

Y'all punks can get the data from kaggle here: https://www.kaggle.com/uciml/sms-spam-collection-dataset/kernels 

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/spam.csv', encoding = 'latin1')

In [2]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
from html import parser

# Drop the empty columns
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)

# Rename the columns
df = df.rename(index=str, columns={"v1": "classification", "v2": "message"})

# Remove HTML encodings
df['message'] = df['message'].apply(parser.unescape)
df.head()

,classification,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['classification'] = pd.get_dummies(df['classification'])

In [5]:
print(df.head())

   classification                                            message
0               1  Go until jurong point, crazy.. Available only ...
1               1                      Ok lar... Joking wif u oni...
2               0  Free entry in 2 a wkly comp to win FA Cup fina...
3               1  U dun say so early hor... U c already then say...
4               1  Nah I don't think he goes to usf, he lives aro...


In [6]:
y = np.array(df['classification'].values.astype(np.int))
np.unique(y)

array([0, 1])

In [28]:
import keras


y = np.array(df['classification'].values)
X = df['message'].values
#print(X)

#y = [0 if y_ == "spam" else 1 for y_ in y]
y_ohe = keras.utils.to_categorical(y)
y_ohe


array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

We need to find how long our longest message is

In [8]:
max = 0
count = 0
for i in X:
    if len(i) > max:
        #if len(i) < 200:
        max = len(i)
        #else:
            #count +=1
        
print(max)
print(count)

910
0


In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
NUM_TOP_WORDS = None

tokenizer = Tokenizer(num_words=NUM_TOP_WORDS)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(X)
sequences = pad_sequences(sequences)
MAX_TEXT_LEN = len(sequences[0])
print(MAX_TEXT_LEN)

189


In [70]:
import numpy as np
from keras.datasets import imdb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    sequences, y, test_size=0.2, stratify=y, random_state=42)


# truncate and pad input sequences to be the same length
max_text_length = 189

NUM_CLASSES = 1

In [12]:
EMBED_SIZE = 100
# the embed size should match the file you load glove from
embeddings_index = {}
f = open('data/glove.6B/glove.6B.100d.txt', encoding="utf8")
# save key/array pairs of the embeddings
#  the key of the dictionary is the word, the array is the embedding
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# now fill in the matrix, using the ordering from the
#  keras word tokenizer from before
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

Found 400000 word vectors.
(8920, 100)


In [13]:
from keras.layers import Embedding



In [84]:
from keras.models import Sequential, Input, Model
from keras.layers import Dense, LSTM, Activation
from keras.regularizers import l2

l2_lambda = 0.0001

input_holder = Input(shape=(189,))
embedding_layer = Embedding(len(word_index) + 1,
                            EMBED_SIZE,
                            weights=[embedding_matrix],
                            input_length=MAX_TEXT_LEN,
                            trainable=False)(input_holder)

lstm_layer = LSTM(100,dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
dense1 = Dense(NUM_CLASSES, 
                activation='sigmoid',
                kernel_initializer='glorot_uniform',
                kernel_regularizer=l2(l2_lambda)
             )(lstm_layer)
#activ1 = Activation("sigmoid")(dense1)

rnn = Model(inputs=input_holder, outputs=dense1)
rnn.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(rnn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 189)               0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 189, 100)          892000    
_________________________________________________________________
lstm_16 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 972,501
Trainable params: 80,501
Non-trainable params: 892,000
_________________________________________________________________
None


In [53]:
for a in y_test:
    print(a)

1
1
1
1
1
0
1
1
1
0
1
0
1
1
1
1
0
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
0
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
0
1
0
1
0
0
1
0
1
1
0
1
1
1
0
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
0
1
1
0
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
0
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1


In [26]:
#y_test = [list(x).index(1.0) for x in list(y_test)] 

TypeError: 'int' object is not iterable

In [85]:
rnn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64)

Train on 4457 samples, validate on 1115 samples
Epoch 1/1
4457/4457 [==============================] - 21s - loss: 0.1771 - acc: 0.9282 - val_loss: 0.1015 - val_acc: 0.9659


In [92]:
yhat = rnn.predict(X_test, verbose=1)
#yhat = np.array([[0,1] if np.argmax(x) == 1 else [1,0] for x in yhat]).astype(float)
#yhat = [list(x).index(1.0) for x in list(yhat)]

#y_test = np.array([[0,1] if np.argmax(x) == 1 else [1,0] for x in y_test]).astype(float)
#y_test = [list(x).index(1.0) for x in list(y_test)]

1115/1115 [==============================] - 2s     


In [93]:
for a in yhat:
    print(a)

[ 0.97467208]
[ 0.98349035]
[ 0.95327336]
[ 0.76417083]
[ 0.97677928]
[ 0.76324522]
[ 0.96209395]
[ 0.9954201]
[ 0.99609488]
[ 0.42066818]
[ 0.96920455]
[ 0.97099429]
[ 0.99460393]
[ 0.98641413]
[ 0.99548113]
[ 0.99416143]
[ 0.53554714]
[ 0.94252485]
[ 0.96262336]
[ 0.99594021]
[ 0.98976797]
[ 0.9972474]
[ 0.06453839]
[ 0.88990241]
[ 0.95513439]
[ 0.70715845]
[ 0.99533564]
[ 0.98690212]
[ 0.996822]
[ 0.99701011]
[ 0.9399448]
[ 0.99744534]
[ 0.88893026]
[ 0.85984635]
[ 0.98941922]
[ 0.95566714]
[ 0.81170267]
[ 0.87695283]
[ 0.98606515]
[ 0.96744519]
[ 0.99840218]
[ 0.98871243]
[ 0.98028654]
[ 0.87874687]
[ 0.99867553]
[ 0.92619729]
[ 0.04536698]
[ 0.99887496]
[ 0.99769944]
[ 0.9946714]
[ 0.98726708]
[ 0.98104525]
[ 0.9891488]
[ 0.98101771]
[ 0.86842918]
[ 0.99749833]
[ 0.98199511]
[ 0.99746883]
[ 0.99029607]
[ 0.9946174]
[ 0.99062258]
[ 0.99734265]
[ 0.9857108]
[ 0.99784267]
[ 0.99012458]
[ 0.65590674]
[ 0.97201759]
[ 0.9114238]
[ 0.99294394]
[ 0.97677928]
[ 0.90182924]
[ 0.85330451]
[ 

[ 0.99714571]
[ 0.27156642]
[ 0.97498]
[ 0.02902172]
[ 0.98883504]
[ 0.04269505]
[ 0.99355906]
[ 0.97891325]
[ 0.99360508]
[ 0.99452704]
[ 0.70291305]
[ 0.03078714]
[ 0.99015057]
[ 0.93412077]
[ 0.19482732]
[ 0.9981786]
[ 0.97166765]
[ 0.95770293]
[ 0.99120355]
[ 0.99562198]
[ 0.04498981]
[ 0.99542433]
[ 0.9937855]
[ 0.98016948]
[ 0.96877813]
[ 0.97643524]
[ 0.99661297]
[ 0.69455904]
[ 0.99691087]
[ 0.9950217]
[ 0.99097508]
[ 0.02449629]
[ 0.97475487]
[ 0.99062997]
[ 0.99758136]
[ 0.99807906]
[ 0.75625843]
[ 0.99574029]
[ 0.08394768]
[ 0.98932403]
[ 0.99740702]
[ 0.99572796]
[ 0.99776506]
[ 0.76734161]
[ 0.090179]
[ 0.99190193]
[ 0.29255018]
[ 0.93478131]
[ 0.96891195]
[ 0.95636964]
[ 0.99794263]
[ 0.9830851]
[ 0.99743277]
[ 0.99431407]
[ 0.99793506]
[ 0.9963941]
[ 0.9823094]
[ 0.97338891]
[ 0.99584997]
[ 0.98803085]
[ 0.99462718]
[ 0.97922587]
[ 0.97914493]
[ 0.99123681]
[ 0.97258806]
[ 0.99728465]
[ 0.23374029]
[ 0.62695467]
[ 0.98850131]
[ 0.27419597]
[ 0.96669948]
[ 0.9949314]
[ 0.

In [87]:
for a in yhat:
    print(a)

[ 0.97467208]
[ 0.98349035]
[ 0.95327336]
[ 0.76417083]
[ 0.97677928]
[ 0.76324522]
[ 0.96209395]
[ 0.9954201]
[ 0.99609488]
[ 0.42066818]
[ 0.96920455]
[ 0.97099429]
[ 0.99460393]
[ 0.98641413]
[ 0.99548113]
[ 0.99416143]
[ 0.53554714]
[ 0.94252485]
[ 0.96262336]
[ 0.99594021]
[ 0.98976797]
[ 0.9972474]
[ 0.06453839]
[ 0.88990241]
[ 0.95513439]
[ 0.70715845]
[ 0.99533564]
[ 0.98690212]
[ 0.996822]
[ 0.99701011]
[ 0.9399448]
[ 0.99744534]
[ 0.88893026]
[ 0.85984635]
[ 0.98941922]
[ 0.95566714]
[ 0.81170267]
[ 0.87695283]
[ 0.98606515]
[ 0.96744519]
[ 0.99840218]
[ 0.98871243]
[ 0.98028654]
[ 0.87874687]
[ 0.99867553]
[ 0.92619729]
[ 0.04536698]
[ 0.99887496]
[ 0.99769944]
[ 0.9946714]
[ 0.98726708]
[ 0.98104525]
[ 0.9891488]
[ 0.98101771]
[ 0.86842918]
[ 0.99749833]
[ 0.98199511]
[ 0.99746883]
[ 0.99029607]
[ 0.9946174]
[ 0.99062258]
[ 0.99734265]
[ 0.9857108]
[ 0.99784267]
[ 0.99012458]
[ 0.65590674]
[ 0.97201759]
[ 0.9114238]
[ 0.99294394]
[ 0.97677928]
[ 0.90182924]
[ 0.85330451]
[ 

[ 0.99560332]
[ 0.99566394]
[ 0.99754721]
[ 0.95962173]
[ 0.99554271]
[ 0.99546683]
[ 0.0394224]
[ 0.31946713]
[ 0.99623895]
[ 0.9840613]
[ 0.99533385]
[ 0.96985787]
[ 0.9952265]
[ 0.99291033]
[ 0.99714571]
[ 0.27156642]
[ 0.97498]
[ 0.02902172]
[ 0.98883504]
[ 0.04269505]
[ 0.99355906]
[ 0.97891325]
[ 0.99360508]
[ 0.99452704]
[ 0.70291305]
[ 0.03078714]
[ 0.99015057]
[ 0.93412077]
[ 0.19482732]
[ 0.9981786]
[ 0.97166765]
[ 0.95770293]
[ 0.99120355]
[ 0.99562198]
[ 0.04498981]
[ 0.99542433]
[ 0.9937855]
[ 0.98016948]
[ 0.96877813]
[ 0.97643524]
[ 0.99661297]
[ 0.69455904]
[ 0.99691087]
[ 0.9950217]
[ 0.99097508]
[ 0.02449629]
[ 0.97475487]
[ 0.99062997]
[ 0.99758136]
[ 0.99807906]
[ 0.75625843]
[ 0.99574029]
[ 0.08394768]
[ 0.98932403]
[ 0.99740702]
[ 0.99572796]
[ 0.99776506]
[ 0.76734161]
[ 0.090179]
[ 0.99190193]
[ 0.29255018]
[ 0.93478131]
[ 0.96891195]
[ 0.95636964]
[ 0.99794263]
[ 0.9830851]
[ 0.99743277]
[ 0.99431407]
[ 0.99793506]
[ 0.9963941]
[ 0.9823094]
[ 0.97338891]
[ 0.99

In [39]:
from sklearn import metrics as mt
rec = mt.recall_score(y_test,yhat)
acc = mt.accuracy_score(y_test,yhat)
print(rec)
print(acc)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [44]:
print(np.array(y_train))

[1 1 1 ..., 1 1 1]


In [41]:
print(yhat)

[[ 0.99228644]
 [ 0.99404907]
 [ 0.990049  ]
 ..., 
 [ 0.99001336]
 [ 0.99323457]
 [ 0.99784422]]
